In [1]:
import sklearn
vers = sklearn.__version__
if vers=='1.0':
  print('OK - we have correct version for skilearn:',vers)
else:
  print('NB! - current version for skilearn:',vers,'.Need upgrade')
  %pip install --upgrade scikit-learn

OK - we have correct version for skilearn: 1.0


# Load data

In [5]:
from sklearn.datasets import fetch_openml

bike_sharing = fetch_openml("Bike_Sharing_Demand", version=2, as_frame=True)
df = bike_sharing.frame

In [6]:
df['CountPercents'] = df["count"] / 1000

In [7]:
df

,season,year,month,hour,holiday,weekday,workingday,weather,temp,feel_temp,humidity,windspeed,count,CountPercents
0,spring,0.0,1.0,0.0,False,6.0,False,clear,9.84,14.395,0.81,0.0000,16.0,0.016
1,spring,0.0,1.0,1.0,False,6.0,False,clear,9.02,13.635,0.80,0.0000,40.0,0.040
2,spring,0.0,1.0,2.0,False,6.0,False,clear,9.02,13.635,0.80,0.0000,32.0,0.032
3,spring,0.0,1.0,3.0,False,6.0,False,clear,9.84,14.395,0.75,0.0000,13.0,0.013
4,spring,0.0,1.0,4.0,False,6.0,False,clear,9.84,14.395,0.75,0.0000,1.0,0.001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,spring,1.0,12.0,19.0,False,1.0,True,misty,10.66,12.880,0.60,11.0014,119.0,0.119
17375,spring,1.0,12.0,20.0,False,1.0,True,misty,10.66,12.880,0.60,11.0014,89.0,0.089
17376,spring,1.0,12.0,21.0,False,1.0,True,clear,10.66,12.880,0.60,11.0014,90.0,0.090
17377,spring,1.0,12.0,22.0,False,1.0,True,clear,10.66,13.635,0.56,8.9981,61.0,0.061


# Split dataset on train-test-val

In [ ]:
from sklearn.model_selection import train_test_split

def splitSklearn(inDF, partVal, partTest) :
  train_val_DF_, test_DF_ = train_test_split(inDF, test_size=partTest)
  train_DF_, val_DF_ = train_test_split(train_val_DF_, test_size=partVal)
  return train_DF_, val_DF_, test_DF_

# https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/
def splitTimeSeries(inDF, partVal, partTest) :
  all_size_ = inDF.shape[0]
  part_Train_ = int( all_size_ * (1-partTest))
  part_Val_ = int( (part_Train_) * (1-partVal))
  train_val_, test_ = inDF[0:part_Train_], inDF[part_Train_:all_size_]
  train_, val_ = train_val_[0:part_Val_], train_val_[part_Val_:all_size_]
  return train_, val_, test_

## 1 version - split sequencial rows of dataset

You can find this suggestion in [Jason Brownlee](https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/)

The suggestion is simple and correct for time-series, because it keeps the time-series structure<br>
**Essence of the idea**:<br>
let's you have dataset with N-rows
*   Choose 1-st K rows as TRAIN dataset 
*   And N-K rows as TEST dataset



In [17]:
partTest=0.2

all_size = df.shape[0]
part_Train = int( all_size * (1-partTest))
train_df, test_df = df[0:part_Train], df[part_Train:]

In [18]:
print(f'Shape DF    ={df.shape}')
print(f'Shape TRAIN ={train_df.shape}')
print(f'Shape TEST  ={test_df.shape}')
print(f'Total lines ={test_df.shape[0]+train_df.shape[0]}')

Shape DF    =(17379, 14)
Shape TRAIN =(13903, 14)
Shape TEST  =(3476, 14)
Total lines =17379


In [13]:
X = train_DF.drop(['count','CountPercents'], axis="columns")
X

,season,year,month,hour,holiday,weekday,workingday,weather,temp,feel_temp,humidity,windspeed
15251,winter,1.0,10.0,16.0,False,2.0,True,rain,25.42,28.030,0.88,8.9981
2993,summer,0.0,5.0,15.0,False,1.0,True,clear,25.42,31.060,0.41,15.0013
10074,spring,1.0,2.0,20.0,False,3.0,True,rain,17.22,21.210,0.88,11.0014
10995,summer,1.0,4.0,7.0,False,0.0,False,clear,15.58,19.695,0.27,11.0014
6852,winter,0.0,10.0,2.0,False,2.0,True,clear,18.04,21.970,0.94,6.0032
...,...,...,...,...,...,...,...,...,...,...,...,...
1262,spring,0.0,2.0,18.0,False,5.0,True,clear,13.12,14.395,0.49,32.9975
453,spring,0.0,1.0,22.0,False,4.0,True,misty,9.84,10.605,0.65,19.0012
6644,winter,0.0,10.0,10.0,False,0.0,False,clear,22.96,26.515,0.68,6.0032
10222,spring,1.0,3.0,0.0,False,3.0,True,clear,11.48,12.880,0.65,19.0012


In [14]:
y = train_DF['CountPercents']
y

15251    0.374
2993     0.199
10074    0.097
10995    0.037
6852     0.005
         ...  
1262     0.157
453      0.056
6644     0.365
10222    0.015
1350     0.078
Name: CountPercents, Length: 13903, dtype: float64

## 2 version - split randomly the dataset

Just use the standard `sklearn.model_selection.train_test_split`

[doc](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

The suggestion is **not correct** for time-series, because it does not keep the time-series structure<br>
The function just split arrays or matrices into random train and test subsets


In [19]:
from sklearn.model_selection import train_test_split

# let's split on train(80%) and test(20% = 0.2)
partTest=0.2
train_DF, test_DF_ = train_test_split(df, test_size=partTest)


In [20]:
print(f'Shape DF    ={df.shape}')
print(f'Shape TRAIN ={train_DF.shape}')
print(f'Shape TEST  ={test_DF_.shape}')
print(f'Total lines ={test_DF_.shape[0]+train_DF.shape[0]}')


Shape DF    =(17379, 14)
Shape TRAIN =(13903, 14)
Shape TEST  =(3476, 14)
Total lines =17379
